In [ ]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses

In [ ]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"


In [ ]:
dataset = tf.keras.utils.get_file('aclImdb_v1',
                                  url,
                                  untar=True,
                                  cache_dir='.',
                                  cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')

In [ ]:
os.listdir(dataset_dir)

['train', 'test', 'README', 'imdbEr.txt', 'imdb.vocab']

In [ ]:
train_dir = os.path.join(dataset_dir,"train")

In [ ]:
os.listdir(train_dir)

['urls_pos.txt',
 'labeledBow.feat',
 'urls_neg.txt',
 'urls_unsup.txt',
 'neg',
 'unsupBow.feat',
 'pos',
 'unsup']

In [ ]:
sample_file = os.path.join(train_dir, "pos/1181_9.txt")

with open(sample_file) as f:
  print(f.read())


Rachel Griffiths writes and directs this award winning short film. A heartwarming story about coping with grief and cherishing the memory of those we've loved and lost. Although, only 15 minutes long, Griffiths manages to capture so much emotion and truth onto film in the short space of time. Bud Tingwell gives a touching performance as Will, a widower struggling to cope with his wife's death. Will is confronted by the harsh reality of loneliness and helplessness as he proceeds to take care of Ruth's pet cow, Tulip. The film displays the grief and responsibility one feels for those they have loved and lost. Good cinematography, great direction, and superbly acted. It will bring tears to all those who have lost a loved one, and survived.


In [ ]:
remove_dir = os.path.join(train_dir, 'unsup')

In [ ]:
shutil.rmtree(remove_dir)

In [ ]:
os.listdir(train_dir)

['urls_pos.txt',
 'labeledBow.feat',
 'urls_neg.txt',
 'urls_unsup.txt',
 'neg',
 'unsupBow.feat',
 'pos']

In [ ]:
batch_size = 32
seed = 42

raw_train_ds = tf.keras.utils.text_dataset_from_directory(train_dir,
                                                          batch_size = batch_size,
                                                          validation_split = 0.2,
                                                          subset = 'training',
                                                          seed = seed 
                                                          )

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
